In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.projections import projection_registry

import project_fitness_function as fit

In [ ]:
file = 'train_data.csv'
terreno_hectares = 2
limite_agua = 8.5
janela_dias = 90
orcamento_maximo = 7.5

interacoes = 250

In [ ]:
df = pd.read_csv(file)

In [ ]:
m2_area = terreno_hectares * 10_000

In [ ]:
best_result = []
# Lista para armazenar os valores de fitness ao longo das iterações
fitness_history = []
# Lista para armazenar os valores médios de fitness ao longo das iterações
avg_fitness_history = []
# Lista para armazenar o desvio padrão de fitness ao longo das iterações
std_fitness_history = []
# Lista para armazenar o melhor fitness encontrado até o momento em cada iteração
best_fitness_so_far = []

def xpto(samples):
    results = []
    for i, sample in enumerate(samples):
        fit_result = fit.project_fitness_function(sample, orcamento_maximo, m2_area, limite_agua, janela_dias)
        results.append({'sample': sample, 'fitness': fit_result})

    results.sort(key=lambda x: x['fitness'],reverse=True)

    if len(best_result) == 0:
        best_result.append(results[0])

    if results[0]['fitness'] > best_result[0]['fitness']:
        best_result[0] = results[0]

    # Calcula a média de fitness desta iteração
    avg_fitness = sum(item['fitness'] for item in results) / len(results)
    # Adiciona à lista de médias
    avg_fitness_history.append(avg_fitness)

    # Calcula o desvio padrão de fitness desta iteração
    fitness_values = [item['fitness'] for item in results]
    std_fitness = np.std(fitness_values)
    # Adiciona à lista de desvios padrão
    std_fitness_history.append(std_fitness)

    # Retorna o melhor fitness desta iteração
    return results[0]['fitness']




# população inicial
initial_population = []

for i in range(100):
    initial_population.append(df.sample(3))

# executa para população inicial
initial_fitness = xpto(initial_population)
fitness_history.append(initial_fitness)
best_fitness_so_far.append(initial_fitness)  # Inicializa com o fitness inicial

#crossover de genes
for i in range(interacoes):
    new_population = fit.crossover(initial_population,df)
    iteration_fitness = xpto(new_population)
    fitness_history.append(iteration_fitness)

    # Atualiza o melhor fitness encontrado até o momento
    current_best = max(best_fitness_so_far[-1], iteration_fitness)
    best_fitness_so_far.append(current_best)

    # Atualiza a população para a próxima iteração
    initial_population = new_population

print(best_result)



In [ ]:

# Plotar o gráfico de melhor aptidão encontrada até o momento
plt.figure(figsize=(10, 6))
plt.plot(range(len(best_fitness_so_far)), best_fitness_so_far, linestyle='-', color='r')
plt.title('Gráfico de Melhor Aptidão Encontrada até o Momento')
plt.xlabel('Iteração')
plt.ylabel('Melhor Fitness')
plt.grid(True)
plt.show()

# Plotar o gráfico de média de aptidão
plt.figure(figsize=(10, 6))
plt.plot(range(len(avg_fitness_history)), avg_fitness_history, linestyle='-', color='g')
plt.title('Gráfico de Média de Aptidão do Algoritmo Genético')
plt.xlabel('Iteração')
plt.ylabel('Média de Fitness')
plt.grid(True)
plt.show()

# Plotar o gráfico de desvio padrão de aptidão
plt.figure(figsize=(10, 6))
plt.plot(range(len(std_fitness_history)), std_fitness_history, linestyle='-', color='m')
plt.title('Gráfico de Desvio Padrão de Aptidão do Algoritmo Genético')
plt.xlabel('Iteração')
plt.ylabel('Desvio Padrão de Fitness')
plt.grid(True)
plt.show()

# Plotar o gráfico de convergência
plt.figure(figsize=(10, 6))
plt.plot(range(len(fitness_history)), fitness_history, linestyle='-', color='b')
plt.title('Gráfico de Convergência do Algoritmo Genético')
plt.xlabel('Iteração')
plt.ylabel('Melhor Fitness')
plt.grid(True)
plt.show()
